Real Time Radar Parameter Optimizer
===================================

Carson Anderson & Calvin Henggeler  
ASEN 5264 Decision Making Under Uncertainty - Spring 2025  
Semester Project  

In [15]:
using POMDPs
using POMDPTools: transition_matrices, reward_vectors, SparseCat, Deterministic, RolloutSimulator, DiscreteBelief, FunctionPolicy, ordered_states, ordered_actions, DiscreteUpdater, UnderlyingMDP
using QuickPOMDPs: QuickPOMDP
using NativeSARSOP: SARSOPSolver
using POMDPTesting: has_consistent_distributions
using QMDP: QMDPSolver
using Plots
using Statistics: std
using POMDPPolicies: alphavectors, FunctionPolicy
using Random, Distributions
include("radarFunctions.jl")
include("RadarDetectAndDecide_Sim.ipynb")

LoadError: LoadError: syntax: { } vector syntax is discontinued around c:\repos\radar-param-optimizer\RadarDetectAndDecide_Sim.ipynb:1
in expression starting at c:\repos\radar-param-optimizer\RadarDetectAndDecide_Sim.ipynb:1

## 1. Create Smart Radar POMDP

In [ ]:
# radar_tracking_calvin = QuickPOMDP(
#     
#     # --- Environment Parameters ---
#     c = 3*10^8  # Speed of light
#     f = 1e9     # Frequency
#     λ = c/f     # Wavelength
#     x_coverage_area = 100e3     # 100 km x 100 km area
#     y_coverage_area = 100e3
#     radar_location = (50e3, 0)  # Radar location (x, y) in meters
# 
#     # --- STATE SPACE ---
#     states = nothing,     # We'll model state generation implicitly
#     
#     # --- ACTION SPACE ---
#     actions = [
#         # Example discretized radar settings (fill in realistic values)
#         # start with a range of differeent realistic settings
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=10),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=10),
#     ],
#     # 
#     
#     obstype = Vector{Float64},   # Observation: [power, phase, delay, doppler]
# 
#     gen = function (s, a, rng)
#         # s = (x, y, vx, vy),   extract sate information
#         x, y, vx, vy = s
#         true_range = sqrt((xp - radar_location[1])^2 + (yp - radar_location[2])^2)
# 
#         #TODO: set noise parameters based on radar settings, then add noise to obsercations measurments
# 
# 
#         # Observation Measurments
#         received_power  = radar_return_power(λ, true_range)
#         view_angle      = radar_view_angle(x, y, radar_location)
#         delay           = radar_return_delay(true_range)
#         doppler_obs     = radar_doppler(s)
#         measurments = [received_power, view_angle, delay, doppler_obs]
# 
#         # solve for the observed state
#         o = radar_meas_2_obs_sate()
# 
#         # REWARD: - (tracking error) - (velocity_error) - (action cost)
#         xo, yo, vxo, vyo = o
#         tracking_error = sqrt((x - xo)^2 + (y - yo)^2)              # Euclidean distance
#         velocity_error = sqrt(vx^2 + vy^2) - sqrt(vxo^2 + vyo^2)    # difference of absolute velocities
#         r = -tracking_error - velocity_error
#         # peanalties for actions should realistically be seen in greater measurement noise
# 
#         # TRANSITION MODEL: Target motion (simple linear)
#         dt = T_ipp * 10     # time step
#         xp = x + vx*dt      # + noise_std*randn(rng)
#         yp = y + vy*dt      # + noise_std*randn(rng)
#                             # keep velocities
#         sp = (xp, yp, vx, vy)
# 
#         return (sp=sp, o=o, r=r)
#     end,
# 
#     # TINITIAL STATE GENERATION
#     # Target starts at either far side edge of the coverage area (x ∈ [0, 100e3], y ∈ [0, 100e3])
#     # target will aways start at y = 50e3, ending at a random y at the other side, gives relative heading
#     # Absolute velocity is uniformy distributed from 50 - 300 m/s, v_x and v_x derived from heading and absolute velocity
#     
#     function generate_initial_target(rng::AbstractRNG=Random.default_rng())
#         # 1. Start on left or right edge
#         start_x = rand(rng, [0.0, 100e3])  # 0 or 100,000 m
#     
#         # 2. Always start at y = 50 km
#         start_y = 50e3  # meters
#     
#         # 3. Pick random end y on the opposite side
#         end_y = rand(rng) * 100e3  # [0, 100_000] m
#     
#         # 4. Compute heading angle from start to end
#         dx = (start_x == 0.0) ? 100e3 : -100e3
#         dy = end_y - start_y
#         heading_rad = atan(dy, dx)  # angle in radians
#     
#         # 5. Choose absolute speed [50, 300] m/s
#         speed = rand(rng, Uniform(50.0, 300.0))
#     
#         # 6. Derive velocity components
#         vx = speed * cos(heading_rad)
#         vy = speed * sin(heading_rad)
#     
#         return (x=start_x, y=start_y, vx=vx, vy=vy)b
#     end
#     
#     initiatial_state generate_initial_target()
# 
#     discount = 0.99,
# 
#     isterminal = s -> false  # Tracking problem — no natural terminal state 
#     # TODO: termainal states will be when the target reaches the end of the radar simulator
# )


In [ ]:
radar = QuickPOMDP(
    
    # ===================
    # --- ENVIRONMENT ---
    # ===================

    # Environment Grid
    x_max_size = 500000
    y_max_size = 500000    
    divisions  = 1000
    x = collect(LinRange(-x_max_size, x_max_size, divisions))
    y = collect(LinRange(-y_max_size, y_max_size, divisions))
    env = RadarEnvironment(x, y)

    # Receiver (same as transmitter)
    pos_rx = SVector(0.0, 0.0)
    snapped_pos_rx = snap_to_grid(env.grid_x, pos_rx)
    rx = Receiver(snapped_pos_rx, Float64[])
    add_receiver!(env, rx)

    # ===================
    # --- STATE SPACE ---
    # ===================
    states = nothing,     # We'll model state generation implicitly
    
    # ====================
    # --- ACTION SPACE ---
    # ====================
    # actions = [:Search, :Guess, :Change], #third action to change parameters of radar settings?

    # Discrete action space with all combinations of parameters
    actions = [(steering_angle, beamwidthm, power) for steering_angle in 0:5:355 for beamwidth in 5:5:45 for power in [1e3, 10e3, 50e3, 100e3] ],
    
    # ========================
    # --- GENERATIVE MODEL ---
    # ========================
    obstype = Vector{Float64},   # Observation: [steering_angle, range, peak_power]

    gen = function(s,a)
        steer_ang, beamwidth, tx_power = a  # Extract action parameters
        x, y = s                            # Extract state parameters

        # Create trnasmitter object with action parameters
        # PointTransmitter(position, frequency, power, tx_time, pulse_width, steering_angle, beamwidth, is_isotropic)
        tx = PointTransmitter(SVector(0.0, 0.0), 1e9, tx_power, 0.0, 0.6, steer_and, beamwidth, false)
        add_transmitter!(env, tx)

        # Act on the Environment
        # ----------------------
        for t = 1:80 
            # perform search
            step!(env, 1.0)
        end
        rm_transmitter!(env, tx)  # Remove the transmitter after use

        # State Transition
        # ------------------
        sp = s
        
        # Reward function
        # --------------- 
        # function guess_reward(s, a)
        #     
        #     # LEVEL 1:
        #     # if power > threshold,
        #         #Detected, Positive reward
        #     # else
        #         # Not detected, Negative reward
        #     # LEVEL 2:
        #     # x_hat, y_hat = solve_position(steering angle, range)
        #     # if x_hat, y_hat close to target position, Positive reward
        #     # else if x_hat, y_hat far from target position, Negative reward
        #     
        # end
        # 
        # if a == :Search
        #     reward = -1.0 
        # elseif a == :Guess
        #     reward = guess_reward(s, a)  # TODO: implement reward function for quessing
        # # elseif a == :Change
        # #    reward = -0.5 
        # end

        # Retreive observation/ measurments
        return_powers_arr = env.receivers[1].received_power .+ randn() * 10
        return_power, return_delay = findmax(return_powers_arr[])
        range = radar_range(return_delay * 10^-3, noise_std = 0.6*10^-6 ) # return time in ms, noise_std in μs

        # Calculate a belif state from noisy observation
        beleif_state = boresight_polar_2_cartesian(range, steer_ang + randn()*beamwidth/2)
    
        tracking_cost = sqrt((x - beleif_state[1])^2 + (y - beleif_state[2])^2)  # Euclidean distance
        power_cost = tx_power  # Cost of power used (arbitrary units)

        # normaize costs to be evenly weighted
        tracking_error = tracking_cost / (sqrt(2 * (x_max_size^2 + y_max_size^2)))  # Normalize to max distance in grid
        power_cost = power_cost / (100e3)  # Normalize to max power in grid

        reward = - (100 * power_cost) - (100 * tracking error) 
        
        return (sp=sp, o=[steering_angle, peak_power, range], r=reward)


    end,

    # ========================
    # INITIAL STATE GENERATION
    # ========================

    initial_state = (x=5e3, y=5e3),  # Initial state of the target
    # Reflector
    pos_ref = SVector(initial_state[1], initial_state[2])
    snapped_pos_ref = snap_to_grid(env.grid_x, pos_ref)
    rx_ref = Reflector(snapped_pos_ref, 0.9, 10^2, false)
    add_reflector!(env, rx_ref)



    discount = 0.99,

    isterminal = s -> false  # Tracking problem — no natural terminal state 
    # TODO: termainal states will be when the target reaches the end of the radar simulator
)


## 2. Create Updater (Particle Filter)